In [1]:
from numpy import zeros, ones, expand_dims
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from keras.models import Model
from keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

In [2]:
def define_discriminator(im_shape):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=im_shape)
    tar_image = Input(shape=im_shape)
    merge = Concatenate()([in_image, tar_image])
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merge)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    d = Activation('sigmoid')(d)
    model = Model([in_image, tar_image], d)
    opt = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [3]:
def encoder_block(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)
    return g

def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g, training=True)
    if dropout:
        g = Dropout(0.5)(g, training=True)
    g = Concatenate()([g, skip_in])
    g = Activation('relu')(g)
    return g

In [4]:
def define_generator(im_shape=(256,256,3)):
    init = RandomNormal(stddev=0.2)
    in_image = Input(shape=im_shape)
    e1 = encoder_block(in_image, 64, batchnorm=False)
    e2 = encoder_block(e1, 128)
    e3 = encoder_block(e2, 256)
    e4 = encoder_block(e3, 512)
    e5 = encoder_block(e4, 512)
    e6 = encoder_block(e5, 512)
    e7 = encoder_block(e6, 512)
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    
    g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    g = Activation('tanh')(g)
    model = Model(in_image, g)
    return model

In [5]:
def define_gan(g_model, d_model, im_shape):
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    in_img = Input(shape=im_shape)
    gen_out = g_model(in_img)
    dis_out = d_model([in_img, gen_out])
    model = Model(in_img, [dis_out, gen_out])
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [6]:
def generate_real_samples(dataset, n_samples, patch_shape):
    trainA, trainB = dataset
    idx = randint(0, trainA.shape[0], n_samples)
    X1, X2 = trainA[idx], trainB[idx]
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return [X1, X2], y

def generate_fake_samples(g_model, samples, patch_shape):
    x = g_model.predict(samples)
    y = zeros((len(x), patch_shape, patch_shape, 1))
    return x, y

In [7]:
def summarize_performance(step, g_model, dataset, n_samples=3):
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    
    for i in range(n_samples):
        plt.subplot(3, n_samples, i+1)
        plt.axis('off')
        plt.title('Source image')
        plt.imshow(X_realA[i])
    for i in range(n_samples):
        plt.subplot(3, 1+n_samples, i+1)
        plt.axis('off')
        plt.title('Generated target image')
        plt.imshow(X_fakeB[i])
    for i in range(n_samples):
        plt.subplot(3, 1+2*n_samples, i+1)
        plt.axis('off')
        plt.title('Real target image')
        plt.imshow(X_realB[i])
    
    filename1 = 'plot_%06d.png' %(step+1)
    plt.savefig(filename1)
    plt.close()
    filename2 = 'g_model_%06d.h5' %(step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' %(filename1, filename2))

In [8]:
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
    n_patch = d_model.output_shape[1]
    trainA, trainB = dataset
    batch_per_epoch = int(len(trainA) / n_batch)
    n_steps = batch_per_epoch * n_epochs
    for i in range(n_steps):
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        if (i+1) % (batch_per_epoch * 10) == 0:
             summarize_performance(i, g_model, dataset)

In [9]:
import os
import numpy as np
from numpy import asarray, load, vstack, savez_compressed
from keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
from datetime import datetime

In [10]:
def load_images(path, size=(256,512)):
    im_list, tar_list = list(), list()
    for filename in os.listdir(path):
        pic = load_img(path + filename, target_size=size)
        pic = img_to_array(pic)
        sat_img, map_img = pic[:, :256], pic[:,256:]
        im_list.append(sat_img)
        tar_list.append(map_img)
    return [asarray(im_list), asarray(tar_list)]

path = '../input/pix2pix-maps/maps/train/'
[src_images, tar_images] = load_images(path)
print('Loaded:', src_images.shape, tar_images.shape)

In [11]:
for i in range(3):
    plt.subplot(2, 3, 1+i)
    plt.axis('off')
    plt.imshow(src_images[i].astype('uint8'))
for i in range(3):
    plt.subplot(2, 3, 1+i+3)
    plt.axis('off')
    plt.imshow(tar_images[i].astype('uint8'))

In [12]:
image_shape = src_images.shape[1:]
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
gan_model = define_gan(g_model, d_model, image_shape)
data = [src_images, tar_images]

In [13]:
def preprocess_data(data):
    X1, X2 = data[0], data[1]
    X1 = X1 - 127.5 / 127.5
    X2 = X2 - 127.5 / 127.5
    return [X1, X2]
dataset = preprocess_data(data)

In [14]:
start_time = datetime.now()
train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1)
stop_time = datetime.now()
print('Execution time:', stop_time - start_time)

In [ ]:
from keras.models import load_model
from numpy.random import randint

In [ ]:
model = load_model('')
def plot_images(src_img, gen_img, tar_img):
    images = vstack((src_img, gen_img, tar_img))
    images = images + 1 / 2.0
    titles = ['Source Image', 'Generated Image', 'Expected Image']
    for i in range(len(images)):
        plt.subplot(1, 3, i+1)
        plt.axis('off')
        plt.imshow(images[i])
        plt.titles(titles[i])
    plt.show()

In [ ]:
[X1, X2] = dataset
idx = randint(0, len(X1), 1)
src_img, tar_img = X1[idx], X2[idx]
gen_img = model.predict(src_img)
plot_images(src_img, gen_img, tar_img)